In [1]:
import sys
import os

import argparse
import requests
from pprint import pprint
import numpy as np

sys.path.append("../")
import predictlol as pl
import config

import json
# from pymongo import MongoClient
import time
import os, sys, errno

api_key = config.api_key

username = 'delphinus6'
region = 'na1'
# match_index = 0

summoner_name, summoner_id, account_id = pl.get_summoner_ids(username, api_key, region)
print(summoner_name, summoner_id, account_id)

delphinus6 78751242 234507298


In [2]:
# get my stats
with open('delphinus6_ranked_solos.json', 'r') as fn:
    ranked_solos = json.load(fn)
pprint(ranked_solos[0])

{'champion': 11,
 'championId': 11,
 'gameCreation': 1532930262077,
 'gameDuration': 1501,
 'gameId': 2835847087,
 'gameMode': 'CLASSIC',
 'gameType': 'MATCHED_GAME',
 'gameVersion': '8.14.239.6309',
 'highestAchievedSeasonTier': 'SILVER',
 'lane': 'JUNGLE',
 'mapId': 11,
 'participantId': 8,
 'participantIdentities': [{'participantId': 1,
                            'player': {'accountId': 200333018,
                                       'currentAccountId': 200333018,
                                       'currentPlatformId': 'NA1',
                                       'matchHistoryUri': '/v1/stats/player_history/NA1/200333018',
                                       'platformId': 'NA1',
                                       'profileIcon': 940,
                                       'summonerId': 37244963,
                                       'summonerName': 'Zeriya'}},
                           {'participantId': 2,
                            'player': {'accountId': 228546193

In [3]:
def average_duration(all_stats):
    return np.mean([x['gameDuration'] for x in all_stats])

average_game_time = average_duration(ranked_solos)
print('%02d:%02d' % (int(np.floor(average_game_time / 60.)), average_game_time % 60))


32:09


In [4]:
def average_kda(all_stats):
    return np.mean([np.array([x['stats']['kills'],x['stats']['deaths'],x['stats']['assists']]) for x in all_stats], axis=0)

k, d, a = average_kda(ranked_solos[0:20])
print('%.2f : %.2f : %.2f' % (k, d, a))
print('Ratio: %.2f' % ((k+a)/d))

10.15 : 6.10 : 4.80
Ratio: 2.45


In [5]:
last_game = ranked_solos[0]

In [6]:
def search_dict_list(dictlist, keys, value):
    # dictlist is a list of dicts
    # keys is a list of strings (in order)
    # value is the value to check against
    if type(keys) is not list:
        if type(keys) is str:
            keys = [keys]
        else:
            sys.exit('Enter valid keys -- string or list!')
    
    matched_dicts = []
    for i, dictionary in enumerate(dictlist):
        dict_value = dictionary.copy()
        for key in keys:
            dict_value = dict_value[key]
        if dict_value == value:
            matched_dicts.append(dictionary)
    return matched_dicts

len(search_dict_list(ranked_solos, ['stats', 'participantId'], 8))
len(search_dict_list(ranked_solos, 'gameId', 2835847087))

1

In [7]:
last_game['lane'], last_game['role']

('JUNGLE', 'NONE')

In [8]:
len(search_dict_list(ranked_solos, ['lane'], 'JUNGLE'))

158

In [9]:
jungle_games = search_dict_list(ranked_solos, ['lane'], 'JUNGLE')
for game in jungle_games[0:1]:
    print()

In [10]:
def search_dict_list(dictlist, keys, value):
    # dictlist is a list of dicts
    # keys is a list of strings (in order)
    # value is the value to check against
    if type(keys) is not list:
        if type(keys) is str:
            keys = [keys]
        else:
            sys.exit('Enter valid keys -- string or list!')
    
    matched_dicts = []
    for i, dictionary in enumerate(dictlist):
        dict_value = dictionary.copy()
        try:
            for key in keys:
                dict_value = dict_value[key]
        except Exception as e:
            pass
        if dict_value == value:
            matched_dicts.append(dictionary)
    return matched_dicts

In [12]:
other_jungler = search_dict_list(last_game['participants'], ['timeline', 'lane'], 'JUNGLE')[0]

In [13]:
my_rate = last_game['stats']['goldEarned'] / last_game['gameDuration']
other_rate = other_jungler['stats']['goldEarned'] / last_game['gameDuration']

print('My rate is %.02f gold/sec' % my_rate)
print('My opponent\'s rate is %.02f gold/sec' % other_rate)

My rate is 9.15 gold/sec
My opponent's rate is 7.06 gold/sec


In [14]:
pl.get_ranking_data(summoner_id, api_key, region='na1')

[{'leagueId': '380fab60-3c63-11e8-80ab-c81f66cf135e',
  'leagueName': "Malphite's Crushers",
  'tier': 'SILVER',
  'queueType': 'RANKED_FLEX_TT',
  'rank': 'III',
  'playerOrTeamId': '78751242',
  'playerOrTeamName': 'delphinus6',
  'leaguePoints': 79,
  'wins': 49,
  'losses': 39,
  'veteran': False,
  'inactive': False,
  'freshBlood': False,
  'hotStreak': True},
 {'leagueId': 'ce469250-6f6d-11e8-b118-c81f66cf2333',
  'leagueName': "Nasus's Spellslingers",
  'tier': 'SILVER',
  'queueType': 'RANKED_SOLO_5x5',
  'rank': 'IV',
  'playerOrTeamId': '78751242',
  'playerOrTeamName': 'delphinus6',
  'leaguePoints': 100,
  'wins': 60,
  'losses': 54,
  'veteran': False,
  'inactive': False,
  'freshBlood': False,
  'hotStreak': True,
  'miniSeries': {'target': 2, 'wins': 0, 'losses': 0, 'progress': 'NNN'}}]